# 🚢 타이타닉 생존자 예측 (상위 10% 목표, 82%+)

타이타닉 생존자 예측 모델링 프로젝트
1. 환경 설정 및 데이터 로드 (Environment Setup & Data Loading)

2. 데이터 탐색 (Exploratory Data Analysis, EDA)

3. 데이터 전처리 (Data Preprocessing) -- **중요!**

4. 피처 엔지니어링 (Feature Engineering)

5. 모델링 및 학습 (Modeling & Training)

6. 모델 평가 및 튜닝 (Evaluation & Tuning)

7. 최종 예측 및 제출 (Prediction & Submission)


## 1. 환경 설정 및 데이터 로드 (Environment Setup & Data Loading)

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
train_df = pd.read_csv('../data/titanic/train.csv')
test_df = pd.read_csv('../data/titanic/test.csv')
combine = [train_df, test_df]

print("데이터 로드 완료!")
train_df.head()

데이터 로드 완료!


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. 데이터 탐색 (Exploratory Data Analysis, EDA)

In [9]:
print(train_df.info())
print("-"*30)
print(train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())
print("-"*30)
print(train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
------------------------------
   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363
------------------------------
      Sex  Survived
0  female  0.742038
1    male  0.188908


## 3. 데이터 전처리 (Data Preprocessing)
- **Title (호칭)** 추출 및 매핑
- **Age (나이)**: 호칭(Title)별 중간값으로 정교하게 채우기
- **Embarked (항구)**: 최빈값 채우기
- **Fare (요금)**: 중간값 채우기

In [10]:
# 1. 호칭(Title) 추출
for dataset in combine:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# 드문 호칭 정리
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

# 숫자로 매핑
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

# 2. 성별 매핑
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# 3. 나이(Age) 결측치 채우기 (Title별 Median 사용)
for dataset in combine:
    for i in range(0, 2): # Sex: 0, 1
        for j in range(1, 6): # Title: 1~5
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Title'] == j)]['Age'].dropna()
            
            # 데이터가 없는 조합(예: 남성+Mrs)은 스킵하여 에러 방지
            if len(guess_df) == 0:
                continue

            age_guess = guess_df.median()
            # 0.5 단위로 반올림
            age_guess = int( age_guess/0.5 + 0.5 ) * 0.5
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Title == j), 'Age'] = age_guess
    
    # 혹시 남은 결측치는 전체 중간값으로 채움
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].median())
    dataset['Age'] = dataset['Age'].astype(int)

# 4. Embarked 채우기
freq_port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# 5. Fare 채우기
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

print("기본 전처리 완료")

기본 전처리 완료


C:\Users\ailee\AppData\Local\Temp\ipykernel_2420\4265364186.py:49: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)


## 4. 피처 엔지니어링 (Feature Engineering)
- **AgeBand, FareBand**: 수치형 데이터를 구간으로(Binning) 나누어 범주화
- **FamilySize, IsAlone**: 가족 유무 파생 변수 생성

In [11]:
# 1. AgeBand 생성 및 변환
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

# 2. FamilySize & IsAlone
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

# 3. FareBand 생성 및 변환
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

# 4. 불필요한 피처 삭제
# AgeBand와 FareBand는 train_df에만 생성했으므로, 삭제 로직을 분리해야 합니다.
common_drop = ['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch']

train_df = train_df.drop(common_drop + ['PassengerId', 'AgeBand', 'FareBand'], axis=1)
test_df = test_df.drop(common_drop, axis=1)

print("최종 데이터 형태:", train_df.shape, test_df.shape)
train_df.head()

최종 데이터 형태: (891, 9) (418, 9)


,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,IsAlone
0,0,3,0,1,0,0,1,2,0
1,1,1,1,2,3,1,3,2,0
2,1,3,1,1,1,0,2,1,1
3,1,1,1,2,3,0,3,2,0
4,0,3,0,2,1,0,1,1,1


## 5. 모델링 및 학습 (Modeling & Training)
- 앙상블 기법(Ensemble) 사용: **Voting Classifier**
- 구성: Random Forest + Gradient Boosting + SVM

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC

# 데이터 준비
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

# 모델 정의 (Best Parameters 적용)
rf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=42)
svc = SVC(probability=True, random_state=42)

# Voting Classifier (Soft Voting)
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('gb', gb), ('svc', svc)],
    voting='soft'
)

# 학습
voting_clf.fit(X_train, Y_train)
print("모델 학습 완료")

모델 학습 완료


## 6. 모델 평가 및 튜닝 (Evaluation & Tuning)
- 교차 검증 (Cross Validation) 수행

In [13]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(voting_clf, X_train, Y_train, cv=5)
print(f"Cross Validation 평균 정확도: {scores.mean():.4f}")

Cross Validation 평균 정확도: 0.8305


## 7. 최종 예측 및 제출 (Prediction & Submission)

In [15]:
final_pred = voting_clf.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": final_pred
})

filename = 'titanic_submission2.csv'
submission.to_csv(filename, index=False)
print(f"제출 파일 저장 완료: {filename}")

제출 파일 저장 완료: titanic_submission2.csv
